# Qulacs & openfermion & psi4 で VQE

必要なもの

- qulacs
- openfermion
- openfermion-psi4
- psi4
- scipy
- numpy

In [1]:
import sys, os
import qulacs

from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.utils import Grid
from openfermion.linalg import get_sparse_operator
from openfermion.chem import MolecularData
from openfermionpsi4 import run_psi4

from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from openfermion.chem import geometry_from_pubchem

methane_geometry = geometry_from_pubchem('Dichlorodifluoromethane')
print(methane_geometry)


[('Cl', (-1.4594, -0.9208, -0.0001)), ('Cl', (1.4592, -0.921, -0.0002)), ('F', (0.0002, 0.8843, 1.0887)), ('F', (0, 0.8847, -1.0884)), ('C', (0, 0.0728, 0))]


In [3]:
from openfermion.chem import MolecularData

# Set parameters to make a simple molecule.
diatomic_bond_length = 1.29
geometry = [('Cl', (-1.4594, -0.9208, -0.0001)), ('Cl', (1.4592, -0.921, -0.0002)), ('F', (0.0002, 0.8843, 1.0887)), ('F', (0, 0.8847, -1.0884)), ('C', (0, 0.0728, 0))]
basis = 'sto-3g'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

# Make molecule and print out a few interesting facts about it.
molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description)
print('Molecule has automatically generated name {}'.format(
    molecule.name))
print('Information about this molecule would be saved at:\n{}\n'.format(
    molecule.filename))
print('This molecule has {} atoms and {} electrons.'.format(
    molecule.n_atoms, molecule.n_electrons))
for atom, atomic_number in zip(molecule.atoms, molecule.protons):
    print('Contains {} atom, which has {} protons.'.format(
        atom, atomic_number))

Molecule has automatically generated name C1-F2-Cl2_sto-3g_singlet_1.29
Information about this molecule would be saved at:
/home/yourn/.conda/envs/qe/lib/python3.9/site-packages/openfermion/testing/data/C1-F2-Cl2_sto-3g_singlet_1.29

This molecule has 5 atoms and 58 electrons.
Contains C atom, which has 6 protons.
Contains F atom, which has 9 protons.
Contains F atom, which has 9 protons.
Contains Cl atom, which has 17 protons.
Contains Cl atom, which has 17 protons.


In [4]:
from openfermion.chem import MolecularData
from openfermionpsi4 import run_psi4

# Set molecule parameters.
basis = '3-21g'
multiplicity = 1
bond_length_interval = 0.2
n_points = 10

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Generate molecule at different bond lengths.
hf_energies = []
fci_energies = []
bond_lengths = []
for point in range(1, n_points + 1):
    bond_length = bond_length_interval * float(point)
    bond_lengths += [bond_length]
    geometry = [('Cl', (-1.4594, -0.9208, -0.0001)), ('Cl', (1.4592, -0.921, -0.0002)), ('F', (0.0002, 0.8843, 1.0887)), ('F', (0, 0.8847, -1.0884)), ('C', (0, 0.0728, 0))]
    molecule = MolecularData(
        geometry, basis, multiplicity,
        description=str(round(bond_length, 2)))
    
    # Run Psi4.
    molecule = run_psi4(molecule,
                        run_scf=run_scf,
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)
    n_qubit = molecule.n_qubits
    n_electron = molecule.n_electrons

    # Print out some results of calculation.
    print('\nAt bond length of {} angstrom, molecular hydrogen flouride has:'.format(
        bond_length))
    print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
    print('MP2 energy of {} Hartree.'.format(molecule.mp2_energy))
    print('FCI energy of {} Hartree.'.format(molecule.fci_energy))
    print('Nuclear repulsion energy between protons is {} Hartree.'.format(
        molecule.nuclear_repulsion))
    for orbital in range(molecule.n_orbitals):
        print('Spatial orbital {} has energy of {} Hartree.'.format(
            orbital, molecule.orbital_energies[orbital]))
    hf_energies += [molecule.hf_energy]
    fci_energies += [molecule.fci_energy]

# Plot.
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(0)
plt.plot(bond_lengths, fci_energies, 'x-')
plt.plot(bond_lengths, hf_energies, 'o-')
plt.ylabel('Energy in Hartree')
plt.xlabel('Bond length in angstrom')
plt.show()

Psi4 calculation for C1-F2-Cl2_3-21g_singlet_0.2 has failed.


KeyboardInterrupt: 

In [ ]:
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

In [ ]:
print(jw_hamiltonian)

In [ ]:
from qulacs import Observable
from qulacs.observable import create_observable_from_openfermion_text
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

In [ ]:
from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import CZ, RY, RZ, merge

depth = n_qubit

In [ ]:
def he_ansatz_circuit(n_qubit, depth, theta_list):
    """he_ansatz_circuit
    Returns hardware efficient ansatz circuit.

    Args:
        n_qubit (:class:`int`):
            the number of qubit used (equivalent to the number of fermionic modes)
        depth (:class:`int`):
            depth of the circuit.
        theta_list (:class:`numpy.ndarray`):
            rotation angles.
    Returns:
        :class:`qulacs.QuantumCircuit`
    """
    circuit = QuantumCircuit(n_qubit)
    for d in range(depth):
        for i in range(n_qubit):
            circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*d]), RZ(i, theta_list[2*i+1+2*n_qubit*d])))
        for i in range(n_qubit//2):
            circuit.add_gate(CZ(2*i, 2*i+1))
        for i in range(n_qubit//2-1):
            circuit.add_gate(CZ(2*i+1, 2*i+2))
    for i in range(n_qubit):
        circuit.add_gate(merge(RY(i, theta_list[2*i+2*n_qubit*depth]), RZ(i, theta_list[2*i+1+2*n_qubit*depth])))

    return circuit

In [ ]:
def cost(theta_list):
    state = QuantumState(n_qubit) #Prepare |00000>
    circuit = he_ansatz_circuit(n_qubit, depth, theta_list) #Construct quantum circuit
    circuit.update_quantum_state(state) #Operate quantum circuit on state
    return qulacs_hamiltonian.get_expectation_value(state) #Calculate expectation value of Hamiltonian


In [ ]:
cost_history = []
init_theta_list = np.random.random(2*n_qubit*(depth+1))*1e-1
cost_history.append(cost(init_theta_list))
method = "BFGS"
options = {"disp": True, "maxiter": 50, "gtol": 1e-6}
opt = minimize(cost, init_theta_list,
               method=method,
               callback=lambda x: cost_history.append(cost(x)))

In [ ]:

plt.rcParams["font.size"] = 18
plt.plot(cost_history, color="red", label="VQE")
plt.plot(range(len(cost_history)), [molecule.fci_energy]*len(cost_history), linestyle="dashed", color="black", label="Exact Solution")
plt.xlabel("Iteration")
plt.ylabel("Energy expectation value")
plt.legend()
plt.show()
